# Iris Classification Pipeline with ZenML

This notebook demonstrates a ZenML pipeline for iris classification, including data loading, model training, evaluation, explainability, and data drift detection.

In [ ]:
!zenml connect --url=https://d13d987c-zenml.cloudinfra.zenml.io

In [1]:
from zenml.client import Client

In [2]:
Client().activate_stack("default")

In [3]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from zenml import step
from zenml import log_artifact_metadata
from typing import Tuple, Dict, Any
from typing_extensions import Annotated

def safe_metadata(data: Any) -> Dict[str, Any]:
    """Create metadata dict with only supported types."""
    metadata = {"shape": data.shape}
    if isinstance(data, pd.DataFrame):
        metadata["columns"] = list(data.columns)
    return metadata

@step
def load_data() -> Tuple[
    Annotated[pd.DataFrame, "X_train"],
    Annotated[pd.DataFrame, "X_test"],
    Annotated[pd.Series, "y_train"],
    Annotated[pd.Series, "y_test"],
]:
    """Load the iris dataset and split into train and test sets."""
    iris = load_iris(as_frame=True)
    X = iris.data
    y = iris.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    for name, data in [("X_train", X_train), ("X_test", X_test), ("y_train", y_train), ("y_test", y_test)]:
        log_artifact_metadata(
            artifact_name=name,
            metadata={"dataset_info": safe_metadata(data)}
        )

    return X_train, X_test, y_train, y_test

In [4]:
import pandas as pd
from sklearn.svm import SVC
from zenml import step, ArtifactConfig
from zenml import log_model_metadata, log_artifact_metadata
from typing_extensions import Annotated

@step(enable_cache=False)
def train_model(
    X_train: pd.DataFrame,
    y_train: pd.Series,
) -> Annotated[SVC, ArtifactConfig(name="model", is_model_artifact=True)]:
    """Train an SVM classifier."""
    model = SVC(kernel='rbf', probability=True)
    model.fit(X_train, y_train)
    train_accuracy = model.score(X_train, y_train)

    log_model_metadata(
        metadata={
            "training_metrics": {
                "train_accuracy": float(train_accuracy),
            },
            "model_info": {
                "model_type": type(model).__name__,
                "kernel": model.kernel,
            }
        }
    )

    log_artifact_metadata(
        artifact_name="model",
        metadata={
            "model_details": {
                "type": type(model).__name__,
                "kernel": model.kernel,
                "n_support": model.n_support_.tolist(),
            }
        }
    )

    return model

In [5]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from zenml import step
from zenml import log_model_metadata, log_artifact_metadata
from typing import Tuple
from typing_extensions import Annotated

@step
def evaluate_model(
    model: SVC,
    X_test: pd.DataFrame,
    y_test: pd.Series,
) -> Tuple[
    Annotated[np.ndarray, "predictions"],
    Annotated[np.ndarray, "probabilities"]
]:
    """Evaluate the model and make predictions."""
    test_accuracy = model.score(X_test, y_test)
    predictions = model.predict(X_test)
    probabilities = model.predict_proba(X_test)

    log_model_metadata(
        metadata={
            "evaluation_metrics": {
                "test_accuracy": float(test_accuracy),
            }
        }
    )

    log_artifact_metadata(
        artifact_name="predictions",
        metadata={
            "prediction_info": {
                "shape": predictions.shape,
                "unique_values": np.unique(predictions).tolist()
            }
        }
    )

    log_artifact_metadata(
        artifact_name="probabilities",
        metadata={
            "probability_info": {
                "shape": probabilities.shape,
                "min": float(np.min(probabilities)),
                "max": float(np.max(probabilities))
            }
        }
    )

    return predictions, probabilities

In [6]:
import pandas as pd
import shap
from sklearn.svm import SVC
from zenml import step
from zenml import log_artifact_metadata
from typing_extensions import Annotated

class SHAPVisualization:
    def __init__(self, shap_values, feature_names):
        self.shap_values = shap_values
        self.feature_names = feature_names

@step
def explain_model(
    model: SVC,
    X_train: pd.DataFrame
) -> Annotated[SHAPVisualization, "shap_visualization"]:
    """Generate SHAP values for model explainability and create a visualization."""
    explainer = shap.KernelExplainer(model.predict_proba, shap.sample(X_train, 100))
    shap_values = explainer.shap_values(X_train.iloc[:100])

    log_artifact_metadata(
        artifact_name="shap_values",
        metadata={
            "shap_info": {
                "shape": [arr.shape for arr in shap_values],
                "n_classes": len(shap_values),
                "n_features": shap_values[0].shape[1],
            }
        }
    )

    return SHAPVisualization(shap_values, X_train.columns)

In [7]:
import pandas as pd
from scipy.stats import ks_2samp
from zenml import step
from zenml import log_artifact_metadata
from typing import Dict
from typing_extensions import Annotated

@step
def detect_data_drift(
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Annotated[Dict[str, float], "drift_metrics"]:
    """Detect data drift between training and test sets."""
    drift_metrics = {}
    for column in X_train.columns:
        _, p_value = ks_2samp(X_train[column], X_test[column])
        drift_metrics[column] = p_value

    log_artifact_metadata(
        artifact_name="drift_metrics",
        metadata={
            "drift_summary": {
                "high_drift_features": [col for col, p in drift_metrics.items() if p < 0.05]
            }
        }
    )

    return drift_metrics

In [8]:
from zenml import pipeline, Model
from zenml.config import DockerSettings

@pipeline(
    enable_cache=True,
    model=Model(name="high_risk_classification")
)
def iris_classification_pipeline():
    X_train, X_test, y_train, y_test = load_data()
    model = train_model(X_train, y_train)
    evaluate_model(model, X_test, y_test)
    explain_model(model, X_train)
    drift_metrics = detect_data_drift(X_train, X_test)

In [9]:
# Run the pipeline
pipeline_run = iris_classification_pipeline()

Initiating a new run for the pipeline: iris_classification_pipeline.
New model version 24 was created.
Dashboard URL for Model Version with name 24 : https://cloud.zenml.io/organizations/fc992c14-d960-4db7-812e-8f070c99c6f0/tenants/939679ed-a10e-453d-8483-e1ac53649d42/model-versions/906ef31d-3a05-426d-8456-158917be69bc
Executing a new run.
Using user: alexej@zenml.io
Using stack: default
  artifact_store: default
  orchestrator: default
Dashboard URL: https://cloud.zenml.io/organizations/fc992c14-d960-4db7-812e-8f070c99c6f0/tenants/939679ed-a10e-453d-8483-e1ac53649d42/runs/0cec5b7e-bbc2-49f3-9b58-c81cb843044a
Using cached version of load_data.
Step load_data has started.
Using cached version of detect_data_drift.
Step detect_data_drift has started.
Caching disabled explicitly for train_model.
Step train_model has started.
Step train_model has finished in 5.014s.
Step train_model completed successfully.
Step evaluate_model has started.
Step evaluate_model has finished in 7.565s.
Step ev

  0%|          | 0/100 [00:00<?, ?it/s]

num_full_subsets = 2
phi = array([0.01593986, 0.01618829, 0.57759973, 0.0285855 ])
phi = array([-0.03420049,  0.00260433, -0.37575721,  0.09701444])
phi = array([ 0.01826062, -0.01879262, -0.20184252, -0.12559993])
num_full_subsets = 2
phi = array([0.00501523, 0.05422835, 0.51849687, 0.04107654])
phi = array([-0.00955895, -0.01912695, -0.34056713,  0.07112186])
phi = array([ 0.00454372, -0.0351014 , -0.17792975, -0.1121984 ])
num_full_subsets = 2
phi = array([-0.01436161, -0.00334595, -0.29735365, -0.01879687])
phi = array([0.03367135, 0.01575862, 0.52697629, 0.06521281])
phi = array([-0.01930974, -0.01241266, -0.22962264, -0.04641594])
num_full_subsets = 2
phi = array([0.02452754, 0.02036571, 0.51721103, 0.05514418])
phi = array([-0.03940273, -0.00655083, -0.32031302,  0.07340865])
phi = array([ 0.01487519, -0.01381488, -0.19689801, -0.12855283])
num_full_subsets = 2
phi = array([0.02360225, 0.0084607 , 0.55587543, 0.04014871])
phi = array([-4.53194406e-02,  1.31507894e-04, -3.4510134

## Run training step on sagemaker

In [10]:
!zenml stack describe 'local-aws-step-operator'
Client().activate_stack('local-aws-step-operator')

⠋ Describing the stack...
⠙ Describing the stack...
              Stack Configuration               
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ COMPONENT_TYPE     │ COMPONENT_NAME          ┃
┠────────────────────┼─────────────────────────┨
┃ ORCHESTRATOR       │ default                 ┃
┠────────────────────┼─────────────────────────┨
┃ STEP_OPERATOR      │ aws-sagemaker-pipelines ┃
┠────────────────────┼─────────────────────────┨
┃ ARTIFACT_STORE     │ aws-sagemaker-pipelines ┃
┠────────────────────┼─────────────────────────┨
┃ CONTAINER_REGISTRY │ aws-sagemaker-pipelines ┃
┠────────────────────┼─────────────────────────┨
┃ IMAGE_BUILDER      │ aws-sagemaker-pipelines ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━┛
        'local-aws-step-operator' stack         
⠙ Describing the stack...
           Labels           
┏━━━━━━━━━━━━━━━━━━┯━━━━━━━┓
┃ LABEL            │ VALUE ┃
┠──────────────────┼───────┨
┃ zenml:full_stack │ True  ┃
┗━━━━━━━━━━━━━━━━━━┷━━━━━━━┛
⠙ Describing 

In [11]:
import pandas as pd
from sklearn.svm import SVC
from zenml import step, ArtifactConfig
from zenml.config import DockerSettings, ResourceSettings
from zenml import log_model_metadata, log_artifact_metadata
from zenml.integrations.aws.flavors.sagemaker_step_operator_flavor import SagemakerStepOperatorSettings
from typing_extensions import Annotated

@step(
    enable_cache=False,
    step_operator="aws-sagemaker-pipelines",
    settings={
        "docker": DockerSettings(python_package_installer="uv", requirements="requirements.txt"),
        "resources": ResourceSettings(memory="32GB"),
        "step_operator.sagemaker": SagemakerStepOperatorSettings(estimator_args={"instance_type": "ml.p3.2xlarge"})
    }
)
def train_model(
    X_train: pd.DataFrame,
    y_train: pd.Series,
) -> Annotated[SVC, ArtifactConfig(name="model", is_model_artifact=True)]:
    """Train an SVM classifier."""
    model = SVC(kernel='rbf', probability=True)
    model.fit(X_train, y_train)
    train_accuracy = model.score(X_train, y_train)

    log_model_metadata(
        metadata={
            "training_metrics": {
                "train_accuracy": float(train_accuracy),
            },
            "model_info": {
                "model_type": type(model).__name__,
                "kernel": model.kernel,
            }
        }
    )

    log_artifact_metadata(
        artifact_name="model",
        metadata={
            "model_details": {
                "type": type(model).__name__,
                "kernel": model.kernel,
                "n_support": model.n_support_.tolist(),
            }
        }
    )

    return model

In [12]:
from zenml import pipeline, Model
from zenml.config import DockerSettings

@pipeline(
    enable_cache=False,
    settings={"docker": DockerSettings(python_package_installer="uv", requirements="requirements.txt")},
    model=Model(name="high_risk_classification")
)
def iris_classification_pipeline():
    X_train, X_test, y_train, y_test = load_data()
    model = train_model(X_train, y_train)
    evaluate_model(model, X_test, y_test)
    explain_model(model, X_train)
    drift_metrics = detect_data_drift(X_train, X_test)

In [13]:
# Run the pipeline
pipeline_run = iris_classification_pipeline()

Initiating a new run for the pipeline: iris_classification_pipeline.
Found credentials in shared credentials file: ~/.aws/credentials
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/xdg-ubuntu/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/alexej/.config/sagemaker/config.yaml
Archiving notebook code...
Could not import Azure service connector: No module named 'azure.identity'.
Could not import HyperAI service connector: No module named 'paramiko'.
Uploading code to s3://zenml-339712793861-483aa118b313/code_uploads/19e346be045005e5ffcd28f15fa2edd1d921045f.tar.gz (Size: 794.00 B).
Code upload finished.
New model version 25 was created.
Dashboard URL for Model Version with name 25 : https://cloud.zenml.io/organizations/fc992c14-d960-4db7-812e-8f070c99c6f0/tenants/939679ed-a10e-453d-8483-e1ac53649d42/model-versions/f00517c2-4236-44c5-bb12-0e6dcfe56625
Unable to find a build to reuse. A previous build can be reused when

  0%|          | 0/100 [00:00<?, ?it/s]

num_full_subsets = 2
phi = array([0.01622625, 0.01649935, 0.57600914, 0.02906267])
phi = array([-0.03370263,  0.00224827, -0.38178407,  0.09716129])
phi = array([ 0.01747638, -0.01874762, -0.19422507, -0.12622396])
num_full_subsets = 2
phi = array([0.00497727, 0.05521629, 0.51591415, 0.04147415])
phi = array([-0.00953692, -0.0200554 , -0.34479667,  0.07122628])
phi = array([ 0.00455965, -0.03516089, -0.17111748, -0.11270042])
num_full_subsets = 2
phi = array([-0.01446083, -0.00340255, -0.29621491, -0.01914446])
phi = array([0.03112158, 0.01533966, 0.53102772, 0.06596056])
phi = array([-0.01666075, -0.01193712, -0.23481282, -0.0468161 ])
num_full_subsets = 2
phi = array([0.02476199, 0.02060921, 0.51461973, 0.05573667])
phi = array([-0.03907597, -0.00683875, -0.32506692,  0.07338417])
phi = array([ 0.01431398, -0.01377046, -0.18955281, -0.12912084])
num_full_subsets = 2
phi = array([0.02397586, 0.00855587, 0.55380216, 0.04073068])
phi = array([-4.47408257e-02,  6.36212934e-06, -3.5096787

PipelineRunResponse(body=PipelineRunResponseBody(created=datetime.datetime(2024, 8, 9, 10, 42, 51), updated=datetime.datetime(2024, 8, 9, 10, 46, 55), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2024, 4, 16, 12, 37, 43), updated=datetime.datetime(2024, 8, 9, 10, 15, 38), active=True, activation_token=None, full_name='Alexej Penner', email_opted_in=True, is_service_account=False, is_admin=False), metadata=None, resources=None, id=UUID('1f821749-2678-45a1-9f04-85a253caa48b'), permission_denied=False, name='alexej@zenml.io'), status=<ExecutionStatus.COMPLETED: 'completed'>, stack=StackResponse(body=StackResponseBody(created=datetime.datetime(2024, 8, 9, 10, 25, 19), updated=datetime.datetime(2024, 8, 9, 10, 25, 19), user=UserResponse(body=UserResponseBody(created=datetime.datetime(2023, 11, 17, 8, 11, 8), updated=datetime.datetime(2024, 8, 9, 10, 8, 51), active=True, activation_token=None, full_name='Stefan Nica', email_opted_in=True, is_service_account=False, is_adm

## Run full training pipeline on sagemaker

In [14]:
!zenml stack describe 'aws-sagemaker-pipelines'
Client().activate_stack('aws-sagemaker-pipelines')

⠋ Describing the stack...
⠙ Describing the stack...
              Stack Configuration               
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ COMPONENT_TYPE     │ COMPONENT_NAME          ┃
┠────────────────────┼─────────────────────────┨
┃ ORCHESTRATOR       │ aws-sagemaker-pipelines ┃
┠────────────────────┼─────────────────────────┨
┃ STEP_OPERATOR      │ aws-sagemaker-pipelines ┃
┠────────────────────┼─────────────────────────┨
┃ ARTIFACT_STORE     │ aws-sagemaker-pipelines ┃
┠────────────────────┼─────────────────────────┨
┃ CONTAINER_REGISTRY │ aws-sagemaker-pipelines ┃
┠────────────────────┼─────────────────────────┨
┃ IMAGE_BUILDER      │ aws-sagemaker-pipelines ┃
┗━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━━━━━━━━┛
        'aws-sagemaker-pipelines' stack         
⠙ Describing the stack...
             Labels             
┏━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━┓
┃ LABEL            │ VALUE     ┃
┠──────────────────┼───────────┨
┃ zenml:deployment │ terraform ┃
┠──────────────────┼──

In [16]:
from zenml import pipeline, Model
from zenml.config import DockerSettings

@pipeline(
    enable_cache=True,
    model=Model(name="high_risk_classification"),
    settings={
        "docker": DockerSettings(python_package_installer="uv", requirements="requirements.txt"),
        "resources": ResourceSettings(memory="8GB"),
    }
)
def iris_classification_pipeline():
    X_train, X_test, y_train, y_test = load_data()
    model = train_model(X_train, y_train)
    evaluate_model(model, X_test, y_test)
    explain_model(model, X_train)
    drift_metrics = detect_data_drift(X_train, X_test)

In [ ]:
# Run the pipeline
pipeline_run = iris_classification_pipeline()

Initiating a new run for the pipeline: iris_classification_pipeline.
Archiving notebook code...
Uploading code to s3://zenml-339712793861-483aa118b313/code_uploads/b88b56da391211690dc82e41ab1ff5b81098c292.tar.gz (Size: 2.06 KiB).
Code upload finished.
New model version 26 was created.
Dashboard URL for Model Version with name 26 : https://cloud.zenml.io/organizations/fc992c14-d960-4db7-812e-8f070c99c6f0/tenants/939679ed-a10e-453d-8483-e1ac53649d42/model-versions/af6ed7ed-7c51-479c-a01f-25e4d6d1b937
Unable to find a build to reuse. A previous build can be reused when the following conditions are met:
  * The existing build was created for the same stack, ZenML version and Python version
  * The stack contains a container registry
  * The Docker settings of the pipeline and all its steps are the same as for the existing build.
Building Docker image(s) for pipeline iris_classification_pipeline.
Building Docker image 339712793861.dkr.ecr.us-east-2.amazonaws.com/zenml-483aa118b313:iris_clas